In [1]:
import numpy as np
import torch as th

device = th.device('cuda' if th.cuda.is_available() else 'cpu')
print('device:', device)

device: cuda


In [3]:
import os

from torch.utils.data import Dataset, DataLoader, random_split

from typing import List

class MovementDataSet(Dataset):
  def __init__(self, data_path: str):
    def load_data(type: str):
      dir_path: str = os.path.join(data_path, type) # ex: data/inputs
      names = []
      values = []
      for f in os.listdir(dir_path):
        names.append(f.replace('.npy', '')) # ex: dash
        values.append(np.load(os.path.join(dir_path, f)).astype(np.float32))
      return names, values
    self.input_names, self.input_values = load_data('input')
    self.output_names, self.output_values = load_data('output')

  def __len__(self):
    return len(self.output_values[0])

  def __getitem__(self, idx):
    def get_line(data):
      line = []
      for v in data:
        line.append(v[idx])
      return tuple(line)
    return get_line(self.input_values), get_line(self.output_values)
    # return tuple(self.input_values[0][idx]), tuple(self.output_values[0][idx])

def print_dataset(name, dataset):
  print('name:', name)
  print('length:', len(dataset))
  # print('dtype:', *[e.dtype for e in dataset[0]])
  print('sample:')
  inputs, outputs = dataset[0]
  print('inputs:')
  for i, (name, value) in enumerate(zip(move_dataset.input_names, inputs)):
    print(name, value, value.shape)
  print('outputs:')
  for i, (name, value) in enumerate(zip(move_dataset.output_names, outputs)):
    print(name, value)
  print()

move_dataset = MovementDataSet('data/only_side_moves')

train_dataset, test_dataset = random_split(move_dataset, [0.7, 0.3], generator=th.Generator().manual_seed(34))

print_dataset('train_dataset', train_dataset)
print_dataset('test_dataset', test_dataset)

name: train_dataset
length: 714
sample:
inputs:
dir [0. 0.] (2,)
vel [ 0.3418274  -0.25506592] (2,)
wall [[1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [4]:
batch_size = 256
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [8]:
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary

th.set_default_dtype(th.float32)

class MyModel(nn.Module):
  def __init__(self):
    super(MyModel, self).__init__()
    self.flatten = nn.Flatten()
    self.wall1 = nn.Linear(18*24, 16)
    self.linear1 = nn.Linear(16+2*2, 16)
    
    self.linear2 = nn.Linear(16+2*2, 2)
    
    # self.linear3 = nn.Linear(18*24+2*3, 2)
    

  def forward(self, dir, vel, wall):
    # x = self.flatten(wall)
    # x = th.cat([dir, jump, vel, x], dim=1)
    # x = F.tanh(self.linear3(x))
    
    x = self.flatten(wall)
    x = F.tanh(self.wall1(x))
    x = th.cat([dir, vel, x], dim = 1)
    x = F.tanh(self.linear1(x))
    x = th.cat([dir, vel, x], dim = 1)
    x = self.linear2(x)
    return (x,)

model = MyModel().to(device)

# summary(model, input_size=(200,), device=device.type)

In [9]:
loss_fn = nn.L1Loss() # mean absolute error loss
optimizer = th.optim.SGD(model.parameters(), lr=0.001, momentum=0.1)

def train_one_epoch(epoch_index, tb_writer):
  running_loss = 0.
  last_loss = 0.

  # Here, we use enumerate(training_loader) instead of
  # iter(training_loader) so that we can track the batch
  # index and do some intra-epoch reporting
  for i, (inputs, labels) in enumerate(train_dataloader):
    # Every data instance is an input + label pair
    inputs = [a.to(device) for a in inputs]
    labels = [a.to(device) for a in labels]
    
    optimizer.zero_grad()

    outputs = model(*inputs)
    
    loss = loss_fn(outputs[0], labels[0])
    loss.backward()

    optimizer.step()

    running_loss += loss.item()
    pc = 2 # print count
    if i % pc == pc-1:
      last_loss = running_loss / pc # loss per batch
      # print('  batch {} loss: {}'.format(i + 1, last_loss))
      # tb_x = epoch_index * len(train_dataloader) + i + 1
      # tb_writer.add_scalar('Loss/train', last_loss, tb_x)
      running_loss = 0.

  return last_loss

In [10]:
from datetime import datetime
from torch.utils.tensorboard.writer import SummaryWriter

# Initializing in a separate cell so we can easily add more epochs to the same run
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/fashion_trainer_{}'.format(timestamp))
epoch_number = 0

EPOCHS = 1000

best_vloss = 1_000_000.

for epoch in range(EPOCHS):
  print('EPOCH {}:'.format(epoch_number + 1))

  # Make sure gradient tracking is on, and do a pass over the data
  model.train(True)
  avg_loss = train_one_epoch(epoch_number, writer)

  # We don't need gradients on to do reporting
  model.train(False)

  running_vloss = 0.0
  for i, (vinputs, vlabels) in enumerate(test_dataloader):
    vinputs = [a.to(device) for a in vinputs]
    vlabels = [a.to(device) for a in vlabels]
    voutputs = model(*vinputs)
    vloss = loss_fn(voutputs[0], vlabels[0])
    running_vloss += vloss

  avg_vloss = running_vloss / (i + 1)
  print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))

  # Log the running loss averaged per batch
  # for both training and validation
  writer.add_scalars('Training vs. Validation Loss',
                  { 'Training' : avg_loss, 'Validation' : avg_vloss },
                  epoch_number + 1)
  writer.flush()

  # Track best performance, and save the model's state
  if avg_vloss < best_vloss:
    best_vloss = avg_vloss
    model_path = 'models/model_{}_{}'.format(timestamp, epoch_number)
    th.save(model.state_dict(), model_path)

  epoch_number += 1

EPOCH 1:
LOSS train 0.25601696968078613 valid 0.22008764743804932
EPOCH 2:
LOSS train 0.25955501943826675 valid 0.22420138120651245
EPOCH 3:
LOSS train 0.24759326875209808 valid 0.20477694272994995
EPOCH 4:
LOSS train 0.2484312206506729 valid 0.24935950338840485
EPOCH 5:
LOSS train 0.24499906599521637 valid 0.22009804844856262
EPOCH 6:
LOSS train 0.23643898963928223 valid 0.23708736896514893
EPOCH 7:
LOSS train 0.23659075051546097 valid 0.21618914604187012
EPOCH 8:
LOSS train 0.23106984794139862 valid 0.21491798758506775
EPOCH 9:
LOSS train 0.22937316447496414 valid 0.21140511333942413
EPOCH 10:
LOSS train 0.24363167583942413 valid 0.20293793082237244
EPOCH 11:
LOSS train 0.23567894846200943 valid 0.20434942841529846
EPOCH 12:
LOSS train 0.2222091406583786 valid 0.2123916894197464
EPOCH 13:
LOSS train 0.21674732118844986 valid 0.21531015634536743
EPOCH 14:
LOSS train 0.223968043923378 valid 0.2320706844329834
EPOCH 15:
LOSS train 0.23070906847715378 valid 0.2086724042892456
EPOCH 16:
L

In [181]:
print(*train_dataset[0][0])
with th.no_grad():
  print(*model(*[a.reshape(1, *a.shape) for a in train_dataset[0][0]]))
  print(*train_dataset[0][1])

[0. 0.] [0. 0.] [ 0.3418274  -0.25506592] [[1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

TypeError: flatten() takes from 0 to 1 positional arguments but 2 were given